In [144]:
import pandas as pd
import spacy
from sklearn.feature_extraction.text import CountVectorizer
import pickle
nlp = spacy.load('en_core_web_sm')

In [104]:
dd = pd.read_csv('anime-dataset-2023.csv')
dd.head(2)

,anime_id,Name,English name,Other name,Score,Genres,Synopsis,Type,Episodes,Aired,...,Studios,Source,Duration,Rating,Rank,Popularity,Favorites,Scored By,Members,Image URL
0,1,Cowboy Bebop,Cowboy Bebop,カウボーイビバップ,8.75,"Action, Award Winning, Sci-Fi","Crime is timeless. By the year 2071, humanity ...",TV,26.0,"Apr 3, 1998 to Apr 24, 1999",...,Sunrise,Original,24 min per ep,R - 17+ (violence & profanity),41.0,43,78525,914193.0,1771505,https://cdn.myanimelist.net/images/anime/4/196...
1,5,Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,カウボーイビバップ 天国の扉,8.38,"Action, Sci-Fi","Another day, another bounty—such is the life o...",Movie,1.0,"Sep 1, 2001",...,Bones,Original,1 hr 55 min,R - 17+ (violence & profanity),189.0,602,1448,206248.0,360978,https://cdn.myanimelist.net/images/anime/1439/...


In [105]:
dd[dd.Name == "One Punch Man"]

,anime_id,Name,English name,Other name,Score,Genres,Synopsis,Type,Episodes,Aired,...,Studios,Source,Duration,Rating,Rank,Popularity,Favorites,Scored By,Members,Image URL
10408,30276,One Punch Man,One Punch Man,ワンパンマン,8.5,"Action, Comedy",The seemingly unimpressive Saitama has a rathe...,TV,12.0,"Oct 5, 2015 to Dec 21, 2015",...,Madhouse,Web manga,24 min per ep,R - 17+ (violence & profanity),127.0,4,62408,2131099.0,3058666,https://cdn.myanimelist.net/images/anime/12/76...


In [106]:
df = df[df.Favorites != 0]
df = df[df.Members >=500]
df[df.Popularity != 0]

,anime_id,Name,English name,Other name,Score,Genres,Synopsis,Type,Episodes,Aired,...,Studios,Source,Duration,Rating,Rank,Popularity,Favorites,Scored By,Members,Image URL
0,1,Cowboy Bebop,Cowboy Bebop,カウボーイビバップ,8.75,"Action, Award Winning, Sci-Fi","Crime is timeless. By the year 2071, humanity ...",TV,26.0,"Apr 3, 1998 to Apr 24, 1999",...,Sunrise,Original,24 min per ep,R - 17+ (violence & profanity),41.0,43,78525,914193.0,1771505,https://cdn.myanimelist.net/images/anime/4/196...
1,5,Cowboy Bebop: Tengoku no Tobira,Cowboy Bebop: The Movie,カウボーイビバップ 天国の扉,8.38,"Action, Sci-Fi","Another day, another bounty—such is the life o...",Movie,1.0,"Sep 1, 2001",...,Bones,Original,1 hr 55 min,R - 17+ (violence & profanity),189.0,602,1448,206248.0,360978,https://cdn.myanimelist.net/images/anime/1439/...
2,6,Trigun,Trigun,トライガン,8.22,"Action, Adventure, Sci-Fi","Vash the Stampede is the man with a $$60,000,0...",TV,26.0,"Apr 1, 1998 to Sep 30, 1998",...,Madhouse,Manga,24 min per ep,PG-13 - Teens 13 or older,328.0,246,15035,356739.0,727252,https://cdn.myanimelist.net/images/anime/7/203...
3,7,Witch Hunter Robin,Witch Hunter Robin,Witch Hunter ROBIN (ウイッチハンターロビン),7.25,"Action, Drama, Mystery, Supernatural",Robin Sena is a powerful craft user drafted in...,TV,26.0,"Jul 3, 2002 to Dec 25, 2002",...,Sunrise,Original,25 min per ep,PG-13 - Teens 13 or older,2764.0,1795,613,42829.0,111931,https://cdn.myanimelist.net/images/anime/10/19...
4,8,Bouken Ou Beet,Beet the Vandel Buster,冒険王ビィト,6.94,"Adventure, Fantasy, Supernatural",It is the dark century and the people are suff...,TV,52.0,"Sep 30, 2004 to Sep 29, 2005",...,Toei Animation,Manga,23 min per ep,PG - Children,4240.0,5126,14,6413.0,15001,https://cdn.myanimelist.net/images/anime/7/215...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
24539,55289,One Piece: Dai Gekisen Tokushuu! Zoro vs. Ooka...,One Piece: Recapping Fierce Fights! Zoro vs. a...,大激戦特集！ゾロ VS 大看板！,6.47,"Action, Adventure, Comedy, Fantasy",This is the 7th recap special of the Wano arc.,Special,1.0,"May 14, 2023",...,Toei Animation,Manga,24 min,PG-13 - Teens 13 or older,6536.0,11725,1,722.0,1367,https://cdn.myanimelist.net/images/anime/1197/...
24549,55306,Telepath,UNKNOWN,テレパス,6.28,UNKNOWN,Music video for the song Telepath by Yorushika...,Music,1.0,"Feb 1, 2023",...,Gainax Kyoto,Original,5 min,PG-13 - Teens 13 or older,UNKNOWN,14301,3,263.0,597,https://cdn.myanimelist.net/images/anime/1339/...
24557,55315,Gensou Mangekyou: The Memories of Phantasm,Touhou Fantasy Kaleidoscope: The Memories of P...,幻想万華鏡 ~The Memories Of Phantasm~,7.78,Fantasy,"Marisa, an ordinary magician, suspects a youka...",OVA,18.0,"Aug 12, 2011 to May 7, 2023",...,UNKNOWN,Game,16 min per ep,PG-13 - Teens 13 or older,963.0,9993,68,662.0,2492,https://cdn.myanimelist.net/images/anime/1729/...
24831,55647,One Piece: Dai Gekisen Tokushuu! Hangeki Doume...,One Piece: Recapping Fierce Fights! The Counte...,大激戦特集！反撃同盟VSビッグ・マム,6.29,"Action, Adventure, Comedy, Fantasy",This is the 8th recap special of the Wano arc.,Special,1.0,"Jun 18, 2023",...,UNKNOWN,Manga,24 min,PG-13 - Teens 13 or older,7480.0,14291,1,320.0,593,https://cdn.myanimelist.net/images/anime/1981/...


In [107]:
df = df[df.Score != "UNKNOWN"]
df.Score = df.Score.astype(float)
df = df[df.Score >= 6]

In [108]:
df.sort_values(by='Popularity')


,anime_id,Name,English name,Other name,Score,Genres,Synopsis,Type,Episodes,Aired,...,Studios,Source,Duration,Rating,Rank,Popularity,Favorites,Scored By,Members,Image URL
7428,16498,Shingeki no Kyojin,Attack on Titan,進撃の巨人,8.54,"Action, Award Winning, Drama, Suspense","Centuries ago, mankind was slaughtered to near...",TV,25.0,"Apr 7, 2013 to Sep 29, 2013",...,Wit Studio,Manga,24 min per ep,R - 17+ (violence & profanity),107.0,1,163844,2660903.0,3744541,https://cdn.myanimelist.net/images/anime/10/47...
1393,1535,Death Note,Death Note,デスノート,8.62,"Supernatural, Suspense","Brutal murders, petty thefts, and senseless vi...",TV,37.0,"Oct 4, 2006 to Jun 27, 2007",...,Madhouse,Manga,23 min per ep,R - 17+ (violence & profanity),74.0,2,167586,2619479.0,3713315,https://cdn.myanimelist.net/images/anime/9/945...
3961,5114,Fullmetal Alchemist: Brotherhood,Fullmetal Alchemist: Brotherhood,鋼の錬金術師 FULLMETAL ALCHEMIST,9.10,"Action, Adventure, Drama, Fantasy",After a horrific alchemy experiment goes wrong...,TV,64.0,"Apr 5, 2009 to Jul 4, 2010",...,Bones,Manga,24 min per ep,R - 17+ (violence & profanity),1.0,3,217606,2020030.0,3176556,https://cdn.myanimelist.net/images/anime/1208/...
10408,30276,One Punch Man,One Punch Man,ワンパンマン,8.50,"Action, Comedy",The seemingly unimpressive Saitama has a rathe...,TV,12.0,"Oct 5, 2015 to Dec 21, 2015",...,Madhouse,Web manga,24 min per ep,R - 17+ (violence & profanity),127.0,4,62408,2131099.0,3058666,https://cdn.myanimelist.net/images/anime/12/76...
6596,11757,Sword Art Online,Sword Art Online,ソードアート・オンライン,7.20,"Action, Adventure, Fantasy, Romance",Ever since the release of the innovative Nerve...,TV,25.0,"Jul 8, 2012 to Dec 23, 2012",...,A-1 Pictures,Light novel,23 min per ep,PG-13 - Teens 13 or older,3055.0,5,67014,2072240.0,2951821,https://cdn.myanimelist.net/images/anime/11/39...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21954,50737,Moshi mo Inochi ga Egaketara,UNKNOWN,もしも命が描けたら,6.74,UNKNOWN,Music video for the song Moshi mo Inochi ga Eg...,Music,1.0,"Dec 21, 2021",...,P.I.C.S.,Other,3 min,PG-13 - Teens 13 or older,UNKNOWN,14900,3,282.0,505,https://cdn.myanimelist.net/images/anime/1391/...
21919,50644,Shijian Qiutu,Time Prisoner,时间囚徒,6.01,"Action, Sci-Fi, Supernatural","In 2060, in order to solve various supernatura...",ONA,12.0,"Oct 3, 2021 to Dec 12, 2021",...,UNKNOWN,Original,14 min per ep,PG-13 - Teens 13 or older,8781.0,14908,1,155.0,504,https://cdn.myanimelist.net/images/anime/1425/...
22239,51375,Hakkoutai Solar Cycle,UNKNOWN,発光体ソーラーサイクル,6.17,UNKNOWN,Music video for the song Hakkoutai Solar Cycle...,Music,1.0,"Mar 24, 2022",...,UNKNOWN,Original,5 min,G - All Ages,UNKNOWN,14916,1,269.0,501,https://cdn.myanimelist.net/images/anime/1116/...
17233,42966,Zhan Shuang Panini,Panini: Gray Raven,战双帕尼尼,6.33,"Action, Comedy","Based on the video game ""Punishing: Gray Raven.""",ONA,12.0,"Oct 27, 2020 to Jan 12, 2021",...,Big Firebird Culture,Game,6 min per ep,PG-13 - Teens 13 or older,7245.0,14925,6,132.0,501,https://cdn.myanimelist.net/images/anime/1045/...


In [109]:
data = df[['anime_id','Name','Genres','Status','Synopsis','Source','Image URL','Type','Rating','Studios','Popularity','Licensors','Producers','Duration']]
data.head()

,anime_id,Name,Genres,Status,Synopsis,Source,Image URL,Type,Rating,Studios,Popularity,Licensors,Producers,Duration
0,1,Cowboy Bebop,"Action, Award Winning, Sci-Fi",Finished Airing,"Crime is timeless. By the year 2071, humanity ...",Original,https://cdn.myanimelist.net/images/anime/4/196...,TV,R - 17+ (violence & profanity),Sunrise,43,"Funimation, Bandai Entertainment",Bandai Visual,24 min per ep
1,5,Cowboy Bebop: Tengoku no Tobira,"Action, Sci-Fi",Finished Airing,"Another day, another bounty—such is the life o...",Original,https://cdn.myanimelist.net/images/anime/1439/...,Movie,R - 17+ (violence & profanity),Bones,602,Sony Pictures Entertainment,"Sunrise, Bandai Visual",1 hr 55 min
2,6,Trigun,"Action, Adventure, Sci-Fi",Finished Airing,"Vash the Stampede is the man with a $$60,000,0...",Manga,https://cdn.myanimelist.net/images/anime/7/203...,TV,PG-13 - Teens 13 or older,Madhouse,246,"Funimation, Geneon Entertainment USA",Victor Entertainment,24 min per ep
3,7,Witch Hunter Robin,"Action, Drama, Mystery, Supernatural",Finished Airing,Robin Sena is a powerful craft user drafted in...,Original,https://cdn.myanimelist.net/images/anime/10/19...,TV,PG-13 - Teens 13 or older,Sunrise,1795,"Funimation, Bandai Entertainment","Bandai Visual, Dentsu, Victor Entertainment, T...",25 min per ep
4,8,Bouken Ou Beet,"Adventure, Fantasy, Supernatural",Finished Airing,It is the dark century and the people are suff...,Manga,https://cdn.myanimelist.net/images/anime/7/215...,TV,PG - Children,Toei Animation,5126,Illumitoon Entertainment,"TV Tokyo, Dentsu",23 min per ep


In [110]:

import re

def remove_newline_numbers(text):
    text = text.replace('\n', ' ')
    text = re.sub(r'\d+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    return text.lower()

In [111]:
data.Synopsis = data.Synopsis.apply(remove_newline_numbers)

C:\Users\91877\AppData\Local\Temp\ipykernel_33180\4278569274.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data.Synopsis = data.Synopsis.apply(remove_newline_numbers)


In [112]:
data.Synopsis.values

array(['crime is timeless by the year  humanity has expanded across the galaxy filling the surface of other planets with settlements like those on earth these new societies are plagued by murder drug use and theft and intergalactic outlaws are hunted by a growing number of tough bounty hunters  spike spiegel and jet black pursue criminals throughout space to make a humble living beneath his goofy and aloof demeanor spike is haunted by the weight of his violent past meanwhile jet manages his own troubled memories while taking care of spike and the bebop their ship the duo is joined by the beautiful con artist faye valentine odd child edward wong hau pepelu tivrusky iv and ein a bioengineered welsh corgi  while developing bonds and working to catch a colorful cast of criminals the bebop crews lives are disrupted by a menace from spikes past as a rivals maniacal plot continues to unravel spike must choose between life with his newfound family or revenge for his old wounds',
       'anothe

In [113]:
data['Synopsis'] = data['Synopsis'].apply(lambda x: " ".join([token.lemma_ for token in nlp(x) if not token.is_stop]))

C:\Users\91877\AppData\Local\Temp\ipykernel_33180\3679476159.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Synopsis'] = data['Synopsis'].apply(lambda x: " ".join([token.lemma_ for token in nlp(x) if not token.is_stop]))


In [114]:
data.Genres.value_counts()

Genres
Hentai                                                      710
Comedy                                                      637
UNKNOWN                                                     482
Action, Adventure, Fantasy                                  276
Action, Sci-Fi                                              262
                                                           ... 
Action, Adventure, Award Winning, Comedy, Drama, Romance      1
Boys Love, Comedy, Sports                                     1
Avant Garde, Drama, Horror                                    1
Award Winning, Comedy, Drama, Fantasy, Mystery                1
Fantasy, Sports                                               1
Name: count, Length: 826, dtype: int64

In [115]:
data = data[data.Genres != "UNKNOWN"]

In [116]:
data = data[data['Image URL'] != 'https://cdn.myanimelist.net/img/sp/icon/apple-touch-icon-256.png']

In [117]:
data = data[data.Synopsis != 'No description available for this anime.']

In [118]:
data.shape

(9143, 14)

In [119]:
data[data.Popularity == 1].Synopsis


7428    century ago mankind slaughter near extinction ...
Name: Synopsis, dtype: object

In [120]:
data['Type'] = data['Type'].replace("UNKNOWN","unknowntype")
data['Source'] = data['Source'].replace("Unknown","unknownsource")
data['Licensors'] = data['Licensors'].replace("UNKNOWN","unknownlicensors")
data['Studios'] = data['Studios'].replace("UNKNOWN","unknownstudios")
data['Rating'] = data['Rating'].replace("UNKNOWN","unknownrating")
data['Producers'] = data['Producers'].replace("UNKNOWN","unknownproducers")

In [121]:
data.Duration.value_counts()

Duration
24 min per ep         1675
23 min per ep         1177
25 min per ep          629
24 min                 201
23 min                 188
                      ... 
1 hr 11 min per ep       1
2 hr 30 min              1
2 hr 48 min              1
2 hr 17 min              1
56 sec per ep            1
Name: count, Length: 254, dtype: int64

In [122]:
def spacing(data):    
    s=" "
    for x in data:
        if x != " ":
            s=s+x
    return s.lower()
        

In [123]:
data.Status = data.Status.apply(spacing)
data.Source = data.Source.apply(spacing)
data.Type = data.Type.apply(spacing)
data.Rating = data.Rating.apply(spacing)
data.Studios = data.Studios.apply(spacing)
data.Licensors = data.Licensors.apply(spacing)
data.Producers = data.Producers.apply(spacing)
data.Duration = data.Duration.apply(spacing)
data.Genres = data.Genres.apply(spacing)

In [124]:
data.Synopsis.values

array(['crime timeless year   humanity expand galaxy fill surface planet settlement like earth new society plague murder drug use theft intergalactic outlaws hunt grow number tough bounty hunter   spike spiegel jet black pursue criminal space humble living beneath goofy aloof demeanor spike haunt weight violent past jet manage troubled memory take care spike bebop ship duo join beautiful con artist faye valentine odd child edward wong hau pepelu tivrusky iv ein bioengineered welsh corgi   develop bond work catch colorful cast criminal bebop crew life disrupt menace spike past rival maniacal plot continue unravel spike choose life newfound family revenge old wound',
       'day bountysuch life unlucky crew bebop routine interrupt faye chase fairly worthless target mar witness oil tanker suddenly explode cause mass hysteria casualty mount strange disease spread smoke blast whopping million woolong price place head suppose perpetrator   life stake solution money problem sight bebop crew s

In [125]:
data['tags'] = data['Synopsis']+data['Genres'] + data['Status'] + data['Source'] + data['Type'] + data['Rating'] + data['Studios']+ data['Licensors']+ data['Producers']+ data['Duration']

In [126]:
data = data[['anime_id','Name','Synopsis','Image URL','Popularity','tags']]

In [127]:
data.tags.values

array(['crime timeless year   humanity expand galaxy fill surface planet settlement like earth new society plague murder drug use theft intergalactic outlaws hunt grow number tough bounty hunter   spike spiegel jet black pursue criminal space humble living beneath goofy aloof demeanor spike haunt weight violent past jet manage troubled memory take care spike bebop ship duo join beautiful con artist faye valentine odd child edward wong hau pepelu tivrusky iv ein bioengineered welsh corgi   develop bond work catch colorful cast criminal bebop crew life disrupt menace spike past rival maniacal plot continue unravel spike choose life newfound family revenge old wound action,awardwinning,sci-fi finishedairing original tv r-17+(violence&profanity) sunrise funimation,bandaientertainment bandaivisual 24minperep',
       'day bountysuch life unlucky crew bebop routine interrupt faye chase fairly worthless target mar witness oil tanker suddenly explode cause mass hysteria casualty mount strange 

In [128]:
data = data.reset_index(drop=True)


In [130]:
data.tail(15)

,anime_id,Name,Synopsis,Image URL,Popularity,tags
9128,54591,"Kimetsu no Yaiba: Jougen Shuuketsu, Soshite Ka...",featurelength cut episode yuukakuhen enter...,https://cdn.myanimelist.net/images/anime/1566/...,2230,featurelength cut episode yuukakuhen enter...
9129,54622,Rakuen Shinshoku: Island of the Dead,description available anime,https://cdn.myanimelist.net/images/anime/1899/...,12117,description available anime hentai currentlyai...
9130,54629,Maid Kyouiku: Botsuraku Kizoku Rurikawa Tsubak...,rurikawa clan fall disgrace daugher clan tsuba...,https://cdn.myanimelist.net/images/anime/1394/...,10305,rurikawa clan fall disgrace daugher clan tsuba...
9131,54656,Buddy Daddies Intermission: Cherry-Pick,recap episode buddy daddy,https://cdn.myanimelist.net/images/anime/1705/...,8115,"recap episode buddy daddy action,comedy fini..."
9132,54738,Majutsushi Orphen Hagure Tabi: Seiiki-hen,choice leave orphen lead winona meet lord immi...,https://cdn.myanimelist.net/images/anime/1873/...,6795,choice leave orphen lead winona meet lord immi...
9133,54761,3-nen Z-gumi Ginpachi-sensei Announcement Special,announcement nen zgumi ginpachisensei,https://cdn.myanimelist.net/images/anime/1218/...,11817,announcement nen zgumi ginpachisensei comedy f...
9134,54772,"Ququ Bucai, Zaixia Yeguai",player dongdong accidentally enter azure world...,https://cdn.myanimelist.net/images/anime/1697/...,9842,player dongdong accidentally enter azure world...
9135,54782,Bio Sekai Meisaku Gekijou,short animation base plot biohazardresident ev...,https://cdn.myanimelist.net/images/anime/1308/...,14038,short animation base plot biohazardresident ev...
9136,54846,Aishang Ta de Liyou,freshman juntang yuan apply minsong university...,https://cdn.myanimelist.net/images/anime/1070/...,6969,freshman juntang yuan apply minsong university...
9137,54851,Yakitori,earth colonize superior alien civilization aki...,https://cdn.myanimelist.net/images/anime/1998/...,10770,earth colonize superior alien civilization aki...


In [131]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')


In [132]:
vector = cv.fit_transform(data['tags']).toarray()

In [133]:
vector.shape

(9143, 5000)

In [134]:
cv.get_feature_names_out()

array(['10min', '10minperep', '11min', ..., 'zombie', 'zone', 'μs'],
      dtype=object)

In [135]:
vector.shape

(9143, 5000)

In [136]:
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vector)

In [137]:
similarity.shape

(9143, 9143)

In [138]:
data[data['Name'] == "One Piece"].index[0]

11

In [139]:
def recommend(anime):
    index = data[data['Name'] == anime].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key= lambda x:x[1])
    for i in distances[1:6]:
        print(data.iloc[i[0]].Name)

In [142]:
recommend("Boku no Hero Academia")

Boku no Hero Academia 2nd Season
Tiger & Bunny
Boku no Hero Academia 4th Season
Boku no Hero Academia 3rd Season
Boku no Hero Academia 6th Season


In [145]:
pickle.dump(data,open('anime.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl','wb'))